In [1]:
import cv2
import numpy as np

In [3]:
def ClickAndCrop(event, x, y, flags, param):     
    global isCropping
    global startPt, endPt
    global image, imageClone    
    
    if event == cv2.EVENT_LBUTTONDOWN:
        imageClone = image.copy()        
        startPt = (x, y)
        isCropping = True
        cv2.imshow("sourceImage", imageClone)
 
    elif event == cv2.EVENT_MOUSEMOVE:
        if isCropping:
            imageClone = image.copy()
            endPt = (x, y)
            cv2.rectangle(imageClone, startPt, endPt, (0, 255, 0), 2)
            cv2.imshow("sourceImage", imageClone)

    elif event == cv2.EVENT_LBUTTONUP:
        imageClone = image.copy()
        endPt = (x, y)
        isCropping = False        
        cv2.rectangle(imageClone, startPt, endPt, (0, 255, 0), 2)
        cv2.imshow("sourceImage", imageClone)

In [9]:
def TrackbarCallBack(a):
    pass

In [ ]:
# TODO trackbar to adjust HSV threshold

In [10]:
def DealWithCropped(imageCropped):
    cv2.namedWindow("croppedImage")    
    cv2.createTrackbar("H", "croppedImage", 5, 10, TrackbarCallBack)
    cv2.createTrackbar("S", "croppedImage", 5, 10, TrackbarCallBack)
    cv2.createTrackbar("V", "croppedImage", 5, 10, TrackbarCallBack)
    
    cv2.imshow("croppedImage", imageCropped)
    

In [11]:
SCALE = 0.3

image = cv2.imread("Pictures/IMG_2933.png")
image = cv2.resize(image, (0,0), fx=SCALE, fy=SCALE, interpolation=cv2.INTER_AREA) 
imageClone = image.copy()

isCropping = False
cv2.namedWindow("sourceImage")
cv2.imshow("sourceImage", image)
cv2.setMouseCallback("sourceImage", ClickAndCrop)

while True:    
    key = cv2.waitKey(0)
    
    if key == ord(' '):    
        print("SpaceBar")
        print("StartPt =", startPt, "EndPt =", endPt)
        imageCropped = image[startPt[1]:endPt[1], startPt[0]:endPt[0]]
        DealWithCropped(imageCropped)
    elif key == 27:
        print("Esc")
        break

cv2.destroyAllWindows()
cv2.waitKey(1)
cv2.destroyAllWindows()

SpaceBar
('StartPt =', (359, 332), 'EndPt =', (648, 581))
Esc


In [2]:
events = [i for i in dir(cv2) if 'EVENT' in i]
print events

['EVENT_FLAG_ALTKEY', 'EVENT_FLAG_CTRLKEY', 'EVENT_FLAG_LBUTTON', 'EVENT_FLAG_MBUTTON', 'EVENT_FLAG_RBUTTON', 'EVENT_FLAG_SHIFTKEY', 'EVENT_LBUTTONDBLCLK', 'EVENT_LBUTTONDOWN', 'EVENT_LBUTTONUP', 'EVENT_MBUTTONDBLCLK', 'EVENT_MBUTTONDOWN', 'EVENT_MBUTTONUP', 'EVENT_MOUSEHWHEEL', 'EVENT_MOUSEMOVE', 'EVENT_MOUSEWHEEL', 'EVENT_RBUTTONDBLCLK', 'EVENT_RBUTTONDOWN', 'EVENT_RBUTTONUP']


In [ ]:
res = cv2.waitKey(0)
print 'You pressed %d (0x%x), LSB: %d (%s)' % (res, res, res % 256,
    repr(chr(res%256)) if res%256 < 128 else '?')
if res == ord(' '):
    print("SpaceBar")
if res == 27:
    print("Esc")